Imports

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from pprint import pprint
from nsepython import *
from datetime import datetime
import time

Get Company Tickers from [NSE Website](https://www1.nseindia.com/content/indices/ind_nifty500list.csv)

In [2]:
ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
tickers_file = pd.read_csv(ticker_url)
tickers_df = tickers_file[['Symbol', 'Company Name']]
tickers = tickers_df['Symbol']
tickers.head()

0      3MINDIA
1          ABB
2          ACC
3       AIAENG
4    APLAPOLLO
Name: Symbol, dtype: object

Scrape Article Headlines and Dates

In [3]:
news_url = 'https://ticker.finology.in/company/'

In [4]:
# list to store article data
data = []
companies_len = len(tickers)
for i in range(270):
    print(i)
    req = Request(url= '{}/{}'.format(news_url, tickers[i]),headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
    response = urlopen(req)
    html = BeautifulSoup(response) 
    news_links = html.select('.newslink')  
    for link in news_links:
        title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        data.append([tickers[i], title, art_date, art_time])  
    '''if (i != 0 and i%200 == 0):
        print('sleeping')
        time.sleep(30)'''
df = pd.DataFrame(data, columns=['Ticker', 'Headline', 'Date', 'Time'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269


In [5]:
if 4//200:
    print('hello')    

In [6]:
pprint(df)

           Ticker                                           Headline  \
0         3MINDIA        3m India informs about postal ballot notice   
1         3MINDIA      3M India informs about trading window closure   
2         3MINDIA  3M India’s promoter to exit PFAS manufacturing...   
3         3MINDIA   3M India informs about loss of share certificate   
4         3MINDIA  3M India informs about issuance of duplicate s...   
...           ...                                                ...   
12165  KALYANKJIL  Kalyan Jewellers India informs about annual se...   
12166  KALYANKJIL             Kalyan Jewell.India - Quaterly Results   
12167  KALYANKJIL             Kalyan Jewell.India - Quaterly Results   
12168  KALYANKJIL             Kalyan Jewell.India - Quaterly Results   
12169  KALYANKJIL  Kalyan Jewellers India informs about non-appli...   

             Date   Time  
0      2022/12/24  12:59  
1      2022/12/23  15:38  
2      2022/12/22  12:27  
3      2022/12/19  17:00  


Sentiment Analysis

In [7]:
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
vader = SentimentIntensityAnalyzer()
scores = df['Headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)

In [9]:
new_df = pd.merge(left=df, right=scores_df, on=df.index.values).drop(['key_0'], axis=1)

In [10]:
new_df

,Ticker,Headline,Date,Time,neg,neu,pos,compound
0,3MINDIA,3m India informs about postal ballot notice,2022/12/24,12:59,0.000,1.000,0.000,0.0000
1,3MINDIA,3M India informs about trading window closure,2022/12/23,15:38,0.000,1.000,0.000,0.0000
2,3MINDIA,3M India’s promoter to exit PFAS manufacturing...,2022/12/22,12:27,0.000,1.000,0.000,0.0000
3,3MINDIA,3M India informs about loss of share certificate,2022/12/19,17:00,0.219,0.571,0.210,-0.0258
4,3MINDIA,3M India informs about issuance of duplicate s...,2022/11/21,16:09,0.000,0.784,0.216,0.2960
...,...,...,...,...,...,...,...,...
12165,KALYANKJIL,Kalyan Jewellers India informs about annual se...,2022/05/11,16:50,0.000,1.000,0.000,0.0000
12166,KALYANKJIL,Kalyan Jewell.India - Quaterly Results,2022/05/11,12:03,0.000,1.000,0.000,0.0000
12167,KALYANKJIL,Kalyan Jewell.India - Quaterly Results,2022/05/11,12:03,0.000,1.000,0.000,0.0000
12168,KALYANKJIL,Kalyan Jewell.India - Quaterly Results,2022/05/11,12:03,0.000,1.000,0.000,0.0000


In [11]:
final_df = new_df.groupby('Ticker').mean()

C:\Users\singh\AppData\Local\Temp\ipykernel_29100\2542017131.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_df = new_df.groupby('Ticker').mean()


In [12]:
final_df

,neg,neu,pos,compound
Ticker,,,,
3MINDIA,0.008760,0.932560,0.058680,0.071982
AARTIDRUGS,0.023298,0.926255,0.050447,0.040130
AAVAS,0.000000,0.973400,0.026600,0.040926
ABB,0.018820,0.912140,0.069040,0.081176
ABBOTINDIA,0.008383,0.926872,0.064723,0.080911
...,...,...,...,...
MAHABANK,0.032340,0.845560,0.122100,0.171672
MAPMYINDIA,0.000000,0.973324,0.026676,0.051451
NAUKRI,0.005420,0.973100,0.021480,0.036470


Get Company Sector, industry data

In [17]:
sector = []
industry = []
mCap = []
for i in range(270):
    print(tickers[i])
    meta = nse_eq(tickers[i])
    print(tickers[i])
    sector.append(meta['industryInfo']['macro'])
    pprint('Sector: {}'.format(meta['industryInfo']['macro']))
    industry.append(meta['industryInfo']['sector'])
    pprint('Industry: {}'.format(meta['industryInfo']['sector']))
    ticker_mcap = round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2)
    mCap.append(ticker_mcap)
    print('market cap is Rs {}'.format(ticker_mcap))
    print('\n')

final_df['sector'] = sector
final_df['industry'] = industry
final_df['mCap (Billion)'] = mCap

3MINDIA
3MINDIA
'Sector: Diversified'
'Industry: Diversified'
market cap is Rs 254.72


ABB
ABB
'Sector: Industrials'
'Industry: Capital Goods'
market cap is Rs 609.23


ACC
ACC
'Sector: Commodities'
'Industry: Construction Materials'
market cap is Rs 440.55


AIAENG
AIAENG
'Sector: Industrials'
'Industry: Capital Goods'
market cap is Rs 232.05


APLAPOLLO
APLAPOLLO
'Sector: Industrials'
'Industry: Capital Goods'
market cap is Rs 316.77


AUBANK
AUBANK
'Sector: Financial Services'
'Industry: Financial Services'
market cap is Rs 418.57


AARTIDRUGS
AARTIDRUGS
'Sector: Healthcare'
'Industry: Healthcare'
market cap is Rs 41.09


AAVAS
AAVAS
'Sector: Financial Services'
'Industry: Financial Services'
market cap is Rs 147.87


ABBOTINDIA
ABBOTINDIA
'Sector: Healthcare'
'Industry: Healthcare'
market cap is Rs 296.5


ADANIENT
ADANIENT
'Sector: Commodities'
'Industry: Metals & Mining'
market cap is Rs 4148.86


ADANIGREEN
ADANIGREEN
'Sector: Utilities'
'Industry: Power'
market cap is Rs 3431.

In [16]:
pprint(nse_eq('APOLLOHOSP'))

{'industryInfo': {'basicIndustry': 'Hospital',
                  'industry': 'Healthcare Services',
                  'macro': 'Healthcare',
                  'sector': 'Healthcare'},
 'info': {'activeSeries': ['EQ'],
          'companyName': 'Apollo Hospitals Enterprise Limited',
          'debtSeries': [],
          'identifier': 'APOLLOHOSPEQN',
          'industry': 'MISCELLANEOUS',
          'isCASec': False,
          'isDebtSec': False,
          'isDelisted': False,
          'isETFSec': False,
          'isFNOSec': True,
          'isSLBSec': True,
          'isSuspended': False,
          'isTop10': False,
          'isin': 'INE437A01024',
          'symbol': 'APOLLOHOSP',
          'tempSuspendedSeries': []},
 'metadata': {'industry': 'Hospital',
              'isin': 'INE437A01024',
              'lastUpdateTime': '18-Jan-2023 12:05:45',
              'listingDate': '10-Jan-1996',
              'pdSectorInd': 'NIFTY '
                             'PHARMA                    

In [18]:
final_df = final_df.reset_index()

In [19]:
final_df

,Ticker,neg,neu,pos,compound,sector,industry,mCap (Billion)
0,3MINDIA,0.008760,0.932560,0.058680,0.071982,Diversified,Diversified,254.72
1,AARTIDRUGS,0.023298,0.926255,0.050447,0.040130,Industrials,Capital Goods,609.23
2,AAVAS,0.000000,0.973400,0.026600,0.040926,Commodities,Construction Materials,440.55
3,ABB,0.018820,0.912140,0.069040,0.081176,Industrials,Capital Goods,232.05
4,ABBOTINDIA,0.008383,0.926872,0.064723,0.080911,Industrials,Capital Goods,316.77
...,...,...,...,...,...,...,...,...
265,MAHABANK,0.032340,0.845560,0.122100,0.171672,Information Technology,Information Technology,192.52
266,MAPMYINDIA,0.000000,0.973324,0.026676,0.051451,Fast Moving Consumer Goods,Fast Moving Consumer Goods,93.79
267,NAUKRI,0.005420,0.973100,0.021480,0.036470,Consumer Discretionary,Consumer Durables,170.42
268,NYKAA,0.004000,0.956380,0.039620,0.067964,Industrials,Construction,79.40


In [20]:
final_df = pd.merge(final_df, tickers_df, left_on='Ticker', right_on='Symbol').drop('Symbol', axis=1)

In [21]:
final_df.columns = ['Symbol', 'Negative', 'Neutral', 'Positive', 'Sentiment Score', 'Sector', 'Industry', 'MCap (Billion)', 'Company Name']

In [22]:
final_df

,Symbol,Negative,Neutral,Positive,Sentiment Score,Sector,Industry,MCap (Billion),Company Name
0,3MINDIA,0.008760,0.932560,0.058680,0.071982,Diversified,Diversified,254.72,3M India Ltd.
1,AARTIDRUGS,0.023298,0.926255,0.050447,0.040130,Industrials,Capital Goods,609.23,Aarti Drugs Ltd.
2,AAVAS,0.000000,0.973400,0.026600,0.040926,Commodities,Construction Materials,440.55,Aavas Financiers Ltd.
3,ABB,0.018820,0.912140,0.069040,0.081176,Industrials,Capital Goods,232.05,ABB India Ltd.
4,ABBOTINDIA,0.008383,0.926872,0.064723,0.080911,Industrials,Capital Goods,316.77,Abbott India Ltd.
...,...,...,...,...,...,...,...,...,...
265,MAHABANK,0.032340,0.845560,0.122100,0.171672,Information Technology,Information Technology,192.52,Bank of Maharashtra
266,MAPMYINDIA,0.000000,0.973324,0.026676,0.051451,Fast Moving Consumer Goods,Fast Moving Consumer Goods,93.79,C.E. Info Systems Ltd.
267,NAUKRI,0.005420,0.973100,0.021480,0.036470,Consumer Discretionary,Consumer Durables,170.42,Info Edge (India) Ltd.
268,NYKAA,0.004000,0.956380,0.039620,0.067964,Industrials,Construction,79.40,FSN E-Commerce Ventures Ltd.


Plotting

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.express as px

In [24]:
fig = px.treemap(
    final_df, path=[px.Constant('Nifty 500'), 'Sector', 'Industry', 'Symbol'], values='MCap (Billion)', color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)
fig.show()

In [25]:
'''
req = Request(url= '{}/{}'.format(news_url, tickers[0]),headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
response = urlopen(req)
html = BeautifulSoup(response)
data = [] 
news_links = html.select('.newslink')
for link in news_links:
    title = link.find('span', class_='h6').text
    date = link.find('small').text
    data.append([title, date])

df = pd.DataFrame(data, columns=['Title', 'Date'])

print(df)
'''

"\nreq = Request(url= '{}/{}'.format(news_url, tickers[0]),headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})\nresponse = urlopen(req)\nhtml = BeautifulSoup(response)\ndata = [] \nnews_links = html.select('.newslink')\nfor link in news_links:\n    title = link.find('span', class_='h6').text\n    date = link.find('small').text\n    data.append([title, date])\n\ndf = pd.DataFrame(data, columns=['Title', 'Date'])\n\nprint(df)\n"

References

https://blog.devgenius.io/best-way-to-speed-up-a-bulk-of-http-requests-in-python-4ec75badabed